# 1. Include Libraries and Check GPU

In [1]:
import os
import wget

import tensorflow as tf

import object_detection
from object_detection.builders import model_builder
from object_detection.protos   import pipeline_pb2
from object_detection.utils    import config_util
from object_detection.utils    import label_map_util
from object_detection.utils    import visualization_utils as viz_utils

from google.protobuf import text_format

# Check whether the GPU is recognised
print(tf.config.list_physical_devices('GPU'))

# Change working directory
os.chdir(os.pardir)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 2. Configure Model

In [2]:
CUSTOM_MODEL_NAME     = 'centernet_hg104_512x512_coco17_tpu-8' 
PRETRAINED_MODEL_NAME = 'centernet_hg104_512x512_coco17_tpu-8'
PRETRAINED_MODEL_URL  = 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME        = 'label_map.pbtxt'

labels = [{'name':'BikeLaneMarker', 'id':1}]

# 3. Create Directories

In [3]:
paths = {
    'WORKSPACE_PATH':        os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH':          os.path.join('Tensorflow', 'scripts'),
    'APIMODEL_PATH':         os.path.join('Tensorflow', 'models'),
    'ANNOTATION_PATH':       os.path.join('Tensorflow', 'workspace', 'annotations'),
    'IMAGE_PATH':            os.path.join('Tensorflow', 'workspace', 'images'),
    'IMAGE_PATH_TRAIN':      os.path.join('Tensorflow', 'workspace', 'images', 'train'),
    'IMAGE_PATH_TEST':      os.path.join('data_sources', 'test1'),
    'IMAGE_PATH_OUT':        os.path.join('data_sources', 'test1_out'),
    'IMAGE_PATH_VIDEO':      os.path.join('data_sources', 'test1_video'),
    #'IMAGE_PATH_TEST':       os.path.join('Tensorflow', 'workspace', 'images', 'test'),
    #'IMAGE_PATH_OUT':        os.path.join('Tensorflow', 'workspace', 'images', 'out'),
    #'IMAGE_PATH_VIDEO':      os.path.join('Tensorflow', 'workspace', 'images', 'video'),
    'MODEL_PATH':            os.path.join('Tensorflow', 'workspace', 'models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace', 'pre-trained-models'),
    'CHECKPOINT_PATH':       os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH':           os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':             os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':           os.path.join('Tensorflow', 'workspace', 'models', CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':           os.path.join('Tensorflow', 'protoc')
}

files = {
    'PIPELINE_CONFIG':  os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP':         os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# Create directories if they do not already exist
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 4. Create Label Map

In [4]:
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 5. Download Selected Pre-trained Model

In [ ]:
# Download pre-trained model (if not already downloaded) and unpack it
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 6. Create Custom Model Config from Defaults

In [5]:
# Copy default model config to our model
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

        1 file(s) copied.


In [6]:
# Show the default model.  Is it SSD or Center_Net or something else?
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

{'model': center_net {
   num_classes: 90
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 512
       max_dimension: 512
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
 },
 'train_config': batch_siz

In [7]:
# Initialise our custom config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [8]:
# Tune number of classes that need to be recognised
# NOTE: Might need to adjust to use either "ssd" or "center_net" etc.
# depending on the type of model printed above
pipeline_config.model.center_net.num_classes = len(labels)

# Tune batch size
pipeline_config.train_config.batch_size = 4

# Fill in paths to labl map and training and validation records
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [9]:
# Write our custom model out to disk
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [10]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [11]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\centernet_hg104_512x512_coco17_tpu-8 --pipeline_config_path=Tensorflow\workspace\models\centernet_hg104_512x512_coco17_tpu-8\pipeline.config --num_train_steps=2000


In [12]:
# !{command}

# 8. Evaluate the Model

In [13]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\centernet_hg104_512x512_coco17_tpu-8 --pipeline_config_path=Tensorflow\workspace\models\centernet_hg104_512x512_coco17_tpu-8\pipeline.config --checkpoint_dir=Tensorflow\workspace\models\centernet_hg104_512x512_coco17_tpu-8


In [14]:
# !{command}

# 9. Load Train Model From Checkpoint

In [15]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Find the latest checkpoint
latest_checkpoint = 'ckpt-1'

checkpoint_files = os.listdir(paths['CHECKPOINT_PATH'])

for f in checkpoint_files:
    if f.startswith('ckpt-'):
        latest_checkpoint = f.split('.')[0]
print('Latest Checkpoint: ' + latest_checkpoint)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], latest_checkpoint)).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

Latest Checkpoint: ckpt-27


# 10. Detect Objects in Test Images

In [16]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [17]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [22]:
# List of selected example files
#IMAGE_FILENAMES = ['24417_-20_270_gsv_0.jpg', '28135_20_180_gsv_0.jpg', '11465_30_270_gsv_0.jpg', '24417_0_0_gsv_0.jpg']

# All test files
IMAGE_FILENAMES = filter(lambda file: file.endswith('.jpg'), os.listdir(paths['IMAGE_PATH_TEST']))

In [23]:
for idx, IMAGE_FILENAME in enumerate(IMAGE_FILENAMES):
    print('{0:04d}: {1:s}'.format(idx, IMAGE_FILENAME))
    
    # Define path for input inmage and output image with detections
    IMAGE_PATH  = os.path.join(paths['IMAGE_PATH_TEST'], IMAGE_FILENAME)
    OUTPUT_PATH = os.path.join(paths['IMAGE_PATH_OUT'],  IMAGE_FILENAME)

    # Read the image and convert it into a tensor
    img = cv2.imread(IMAGE_PATH)
    image_np = np.array(img)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    
    # Detect objects of interest using the model
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # Create a copy of the image with detection boxes overlaid
    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

    # Convert color space of the output image
    image_np_converted = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)

    # Write the output image to disk
    cv2.imwrite(OUTPUT_PATH, image_np_with_detections)

    # Display the output image
    #plt.imshow(image_np_converted)
    #plt.show()

0000: gsv_1323522473_-10_149_gsv_0.jpg
Instructions for updating:
Use `tf.cast` instead.
0001: gsv_1323522473_-10_239_gsv_0.jpg
0002: gsv_1323522473_-10_329_gsv_0.jpg
0003: gsv_1323522473_-10_59_gsv_0.jpg
0004: gsv_1323522473_-20_149_gsv_0.jpg
0005: gsv_1323522473_-20_239_gsv_0.jpg
0006: gsv_1323522473_-20_329_gsv_0.jpg
0007: gsv_1323522473_-20_59_gsv_0.jpg
0008: gsv_1323522473_0_149_gsv_0.jpg
0009: gsv_1323522473_0_239_gsv_0.jpg
0010: gsv_1323522473_0_329_gsv_0.jpg
0011: gsv_1323522473_0_59_gsv_0.jpg
0012: gsv_1323522476_-10_134_gsv_0.jpg
0013: gsv_1323522476_-10_224_gsv_0.jpg
0014: gsv_1323522476_-10_314_gsv_0.jpg
0015: gsv_1323522476_-10_44_gsv_0.jpg
0016: gsv_1323522476_-20_134_gsv_0.jpg
0017: gsv_1323522476_-20_224_gsv_0.jpg
0018: gsv_1323522476_-20_314_gsv_0.jpg
0019: gsv_1323522476_-20_44_gsv_0.jpg
0020: gsv_1323522476_0_134_gsv_0.jpg
0021: gsv_1323522476_0_224_gsv_0.jpg
0022: gsv_1323522476_0_314_gsv_0.jpg
0023: gsv_1323522476_0_44_gsv_0.jpg
0024: gsv_1323522496_-10_148_gsv_0.j

0210: gsv_1470081065_10_98_gsv_0.jpg
0211: gsv_1470081065_20_107_gsv_0.jpg
0212: gsv_1470081065_20_17_gsv_0.jpg
0213: gsv_1470081065_20_188_gsv_0.jpg
0214: gsv_1470081065_20_197_gsv_0.jpg
0215: gsv_1470081065_20_278_gsv_0.jpg
0216: gsv_1470081065_20_287_gsv_0.jpg
0217: gsv_1470081065_20_8_gsv_0.jpg
0218: gsv_1470081065_20_98_gsv_0.jpg
0219: gsv_1832927607_0_158_gsv_0.jpg
0220: gsv_1832927607_0_248_gsv_0.jpg
0221: gsv_1832927607_0_338_gsv_0.jpg
0222: gsv_1832927607_0_68_gsv_0.jpg
0223: gsv_1832927607_10_158_gsv_0.jpg
0224: gsv_1832927607_10_248_gsv_0.jpg
0225: gsv_1832927607_10_338_gsv_0.jpg
0226: gsv_1832927607_10_68_gsv_0.jpg
0227: gsv_1832927607_20_158_gsv_0.jpg
0228: gsv_1832927607_20_248_gsv_0.jpg
0229: gsv_1832927607_20_338_gsv_0.jpg
0230: gsv_1832927607_20_68_gsv_0.jpg
0231: gsv_1832927621_0_148_gsv_0.jpg
0232: gsv_1832927621_0_238_gsv_0.jpg
0233: gsv_1832927621_0_328_gsv_0.jpg
0234: gsv_1832927621_0_58_gsv_0.jpg
0235: gsv_1832927621_10_148_gsv_0.jpg
0236: gsv_1832927621_10_238_g

0432: gsv_3110272748_-20_207_gsv_0.jpg
0433: gsv_3110272748_-20_27_gsv_0.jpg
0434: gsv_3110272748_-20_297_gsv_0.jpg
0435: gsv_3110272748_0_117_gsv_0.jpg
0436: gsv_3110272748_0_207_gsv_0.jpg
0437: gsv_3110272748_0_27_gsv_0.jpg
0438: gsv_3110272748_0_297_gsv_0.jpg
0439: gsv_3110272748_10_119_gsv_0.jpg
0440: gsv_3110272748_10_209_gsv_0.jpg
0441: gsv_3110272748_10_299_gsv_0.jpg
0442: gsv_3110272748_10_29_gsv_0.jpg
0443: gsv_3110272748_20_119_gsv_0.jpg
0444: gsv_3110272748_20_209_gsv_0.jpg
0445: gsv_3110272748_20_299_gsv_0.jpg
0446: gsv_3110272748_20_29_gsv_0.jpg
0447: gsv_3110274994_-10_118_gsv_0.jpg
0448: gsv_3110274994_-10_208_gsv_0.jpg
0449: gsv_3110274994_-10_28_gsv_0.jpg
0450: gsv_3110274994_-10_298_gsv_0.jpg
0451: gsv_3110274994_-20_118_gsv_0.jpg
0452: gsv_3110274994_-20_208_gsv_0.jpg
0453: gsv_3110274994_-20_28_gsv_0.jpg
0454: gsv_3110274994_-20_298_gsv_0.jpg
0455: gsv_3110274994_0_118_gsv_0.jpg
0456: gsv_3110274994_0_208_gsv_0.jpg
0457: gsv_3110274994_0_28_gsv_0.jpg
0458: gsv_31102

0653: gsv_367133496_-20_237_gsv_0.jpg
0654: gsv_367133496_-20_238_gsv_0.jpg
0655: gsv_367133496_-20_327_gsv_0.jpg
0656: gsv_367133496_-20_328_gsv_0.jpg
0657: gsv_367133496_-20_57_gsv_0.jpg
0658: gsv_367133496_-20_58_gsv_0.jpg
0659: gsv_367133496_0_145_gsv_0.jpg
0660: gsv_367133496_0_148_gsv_0.jpg
0661: gsv_367133496_0_235_gsv_0.jpg
0662: gsv_367133496_0_238_gsv_0.jpg
0663: gsv_367133496_0_325_gsv_0.jpg
0664: gsv_367133496_0_328_gsv_0.jpg
0665: gsv_367133496_0_55_gsv_0.jpg
0666: gsv_367133496_0_58_gsv_0.jpg
0667: gsv_367133496_10_142_gsv_0.jpg
0668: gsv_367133496_10_232_gsv_0.jpg
0669: gsv_367133496_10_322_gsv_0.jpg
0670: gsv_367133496_10_52_gsv_0.jpg
0671: gsv_367133496_20_142_gsv_0.jpg
0672: gsv_367133496_20_232_gsv_0.jpg
0673: gsv_367133496_20_322_gsv_0.jpg
0674: gsv_367133496_20_52_gsv_0.jpg
0675: gsv_367133524_-10_100_gsv_0.jpg
0676: gsv_367133524_-10_10_gsv_0.jpg
0677: gsv_367133524_-10_190_gsv_0.jpg
0678: gsv_367133524_-10_280_gsv_0.jpg
0679: gsv_367133524_-20_100_gsv_0.jpg
0680:

0876: gsv_367133540_10_7_gsv_0.jpg
0877: gsv_367133540_10_93_gsv_0.jpg
0878: gsv_367133540_10_97_gsv_0.jpg
0879: gsv_367133540_20_183_gsv_0.jpg
0880: gsv_367133540_20_187_gsv_0.jpg
0881: gsv_367133540_20_273_gsv_0.jpg
0882: gsv_367133540_20_277_gsv_0.jpg
0883: gsv_367133540_20_3_gsv_0.jpg
0884: gsv_367133540_20_7_gsv_0.jpg
0885: gsv_367133540_20_93_gsv_0.jpg
0886: gsv_367133540_20_97_gsv_0.jpg
0887: gsv_367133541_-10_187_gsv_0.jpg
0888: gsv_367133541_-10_277_gsv_0.jpg
0889: gsv_367133541_-10_7_gsv_0.jpg
0890: gsv_367133541_-10_97_gsv_0.jpg
0891: gsv_367133541_-20_187_gsv_0.jpg
0892: gsv_367133541_-20_277_gsv_0.jpg
0893: gsv_367133541_-20_7_gsv_0.jpg
0894: gsv_367133541_-20_97_gsv_0.jpg
0895: gsv_367133541_0_188_gsv_0.jpg
0896: gsv_367133541_0_189_gsv_0.jpg
0897: gsv_367133541_0_278_gsv_0.jpg
0898: gsv_367133541_0_279_gsv_0.jpg
0899: gsv_367133541_0_8_gsv_0.jpg
0900: gsv_367133541_0_98_gsv_0.jpg
0901: gsv_367133541_0_99_gsv_0.jpg
0902: gsv_367133541_0_9_gsv_0.jpg
0903: gsv_367133541_10_

...

1548: gsv_458581783_10_114_gsv_0.jpg
1549: gsv_458581783_10_16_gsv_0.jpg
1550: gsv_458581783_10_196_gsv_0.jpg
1551: gsv_458581783_10_204_gsv_0.jpg
1552: gsv_458581783_10_24_gsv_0.jpg
1553: gsv_458581783_10_286_gsv_0.jpg
1554: gsv_458581783_10_294_gsv_0.jpg
1555: gsv_458581783_20_106_gsv_0.jpg
1556: gsv_458581783_20_114_gsv_0.jpg
1557: gsv_458581783_20_16_gsv_0.jpg
1558: gsv_458581783_20_196_gsv_0.jpg
1559: gsv_458581783_20_204_gsv_0.jpg
1560: gsv_458581783_20_24_gsv_0.jpg
1561: gsv_458581783_20_286_gsv_0.jpg
1562: gsv_458581783_20_294_gsv_0.jpg
1563: gsv_458581794_-10_110_gsv_0.jpg
1564: gsv_458581794_-10_200_gsv_0.jpg
1565: gsv_458581794_-10_20_gsv_0.jpg
1566: gsv_458581794_-10_290_gsv_0.jpg
1567: gsv_458581794_-20_110_gsv_0.jpg
1568: gsv_458581794_-20_200_gsv_0.jpg
1569: gsv_458581794_-20_20_gsv_0.jpg
1570: gsv_458581794_-20_290_gsv_0.jpg
1571: gsv_458581794_0_107_gsv_0.jpg
1572: gsv_458581794_0_110_gsv_0.jpg
1573: gsv_458581794_0_17_gsv_0.jpg
1574: gsv_458581794_0_197_gsv_0.jpg
1575

1772: gsv_458581856_0_124_gsv_0.jpg
1773: gsv_458581856_0_212_gsv_0.jpg
1774: gsv_458581856_0_214_gsv_0.jpg
1775: gsv_458581856_0_302_gsv_0.jpg
1776: gsv_458581856_0_304_gsv_0.jpg
1777: gsv_458581856_0_32_gsv_0.jpg
1778: gsv_458581856_0_34_gsv_0.jpg
1779: gsv_458581856_10_122_gsv_0.jpg
1780: gsv_458581856_10_212_gsv_0.jpg
1781: gsv_458581856_10_302_gsv_0.jpg
1782: gsv_458581856_10_32_gsv_0.jpg
1783: gsv_458581856_20_122_gsv_0.jpg
1784: gsv_458581856_20_212_gsv_0.jpg
1785: gsv_458581856_20_302_gsv_0.jpg
1786: gsv_458581856_20_32_gsv_0.jpg
1787: gsv_458581859_-10_106_gsv_0.jpg
1788: gsv_458581859_-10_117_gsv_0.jpg
1789: gsv_458581859_-10_16_gsv_0.jpg
1790: gsv_458581859_-10_196_gsv_0.jpg
1791: gsv_458581859_-10_207_gsv_0.jpg
1792: gsv_458581859_-10_27_gsv_0.jpg
1793: gsv_458581859_-10_286_gsv_0.jpg
1794: gsv_458581859_-10_297_gsv_0.jpg
1795: gsv_458581859_-20_106_gsv_0.jpg
1796: gsv_458581859_-20_117_gsv_0.jpg
1797: gsv_458581859_-20_16_gsv_0.jpg
1798: gsv_458581859_-20_196_gsv_0.jpg
179

1994: gsv_607648588_-20_61_gsv_0.jpg
1995: gsv_607648588_0_149_gsv_0.jpg
1996: gsv_607648588_0_150_gsv_0.jpg
1997: gsv_607648588_0_239_gsv_0.jpg
1998: gsv_607648588_0_240_gsv_0.jpg
1999: gsv_607648588_0_329_gsv_0.jpg
2000: gsv_607648588_0_330_gsv_0.jpg
2001: gsv_607648588_0_59_gsv_0.jpg
2002: gsv_607648588_0_60_gsv_0.jpg
2003: gsv_607648588_10_149_gsv_0.jpg
2004: gsv_607648588_10_239_gsv_0.jpg
2005: gsv_607648588_10_329_gsv_0.jpg
2006: gsv_607648588_10_59_gsv_0.jpg
2007: gsv_607648588_20_149_gsv_0.jpg
2008: gsv_607648588_20_239_gsv_0.jpg
2009: gsv_607648588_20_329_gsv_0.jpg
2010: gsv_607648588_20_59_gsv_0.jpg
2011: gsv_607649480_-10_148_gsv_0.jpg
2012: gsv_607649480_-10_238_gsv_0.jpg
2013: gsv_607649480_-10_328_gsv_0.jpg
2014: gsv_607649480_-10_58_gsv_0.jpg
2015: gsv_607649480_-20_148_gsv_0.jpg
2016: gsv_607649480_-20_238_gsv_0.jpg
2017: gsv_607649480_-20_328_gsv_0.jpg
2018: gsv_607649480_-20_58_gsv_0.jpg
2019: gsv_607649480_0_148_gsv_0.jpg
2020: gsv_607649480_0_238_gsv_0.jpg
2021: gsv

2218: gsv_638346186_20_59_gsv_0.jpg
2219: gsv_638346191_-10_148_gsv_0.jpg
2220: gsv_638346191_-10_238_gsv_0.jpg
2221: gsv_638346191_-10_328_gsv_0.jpg
2222: gsv_638346191_-10_58_gsv_0.jpg
2223: gsv_638346191_-20_148_gsv_0.jpg
2224: gsv_638346191_-20_238_gsv_0.jpg
2225: gsv_638346191_-20_328_gsv_0.jpg
2226: gsv_638346191_-20_58_gsv_0.jpg
2227: gsv_638346191_0_148_gsv_0.jpg
2228: gsv_638346191_0_238_gsv_0.jpg
2229: gsv_638346191_0_328_gsv_0.jpg
2230: gsv_638346191_0_58_gsv_0.jpg
2231: gsv_638346191_10_148_gsv_0.jpg
2232: gsv_638346191_10_238_gsv_0.jpg
2233: gsv_638346191_10_328_gsv_0.jpg
2234: gsv_638346191_10_58_gsv_0.jpg
2235: gsv_638346191_20_148_gsv_0.jpg
2236: gsv_638346191_20_238_gsv_0.jpg
2237: gsv_638346191_20_328_gsv_0.jpg
2238: gsv_638346191_20_58_gsv_0.jpg
2239: gsv_638346230_-10_148_gsv_0.jpg
2240: gsv_638346230_-10_238_gsv_0.jpg
2241: gsv_638346230_-10_328_gsv_0.jpg
2242: gsv_638346230_-10_58_gsv_0.jpg
2243: gsv_638346230_-20_148_gsv_0.jpg
2244: gsv_638346230_-20_238_gsv_0.jp

2442: gsv_768939447_20_190_gsv_0.jpg
2443: gsv_768939447_20_201_gsv_0.jpg
2444: gsv_768939447_20_21_gsv_0.jpg
2445: gsv_768939447_20_280_gsv_0.jpg
2446: gsv_768939447_20_291_gsv_0.jpg
2447: gsv_768939466_-10_111_gsv_0.jpg
2448: gsv_768939466_-10_121_gsv_0.jpg
2449: gsv_768939466_-10_201_gsv_0.jpg
2450: gsv_768939466_-10_211_gsv_0.jpg
2451: gsv_768939466_-10_21_gsv_0.jpg
2452: gsv_768939466_-10_291_gsv_0.jpg
2453: gsv_768939466_-10_301_gsv_0.jpg
2454: gsv_768939466_-10_31_gsv_0.jpg
2455: gsv_768939466_-20_111_gsv_0.jpg
2456: gsv_768939466_-20_121_gsv_0.jpg
2457: gsv_768939466_-20_201_gsv_0.jpg
2458: gsv_768939466_-20_211_gsv_0.jpg
2459: gsv_768939466_-20_21_gsv_0.jpg
2460: gsv_768939466_-20_291_gsv_0.jpg
2461: gsv_768939466_-20_301_gsv_0.jpg
2462: gsv_768939466_-20_31_gsv_0.jpg
2463: gsv_768939466_0_111_gsv_0.jpg
2464: gsv_768939466_0_121_gsv_0.jpg
2465: gsv_768939466_0_201_gsv_0.jpg
2466: gsv_768939466_0_211_gsv_0.jpg
2467: gsv_768939466_0_21_gsv_0.jpg
2468: gsv_768939466_0_291_gsv_0.j

# 11. Detect Objects in Video

In [ ]:
VIDEO_FILE_IN  = os.path.join(paths['IMAGE_PATH_VIDEO'], '1.mp4')
VIDEO_FILE_OUT = os.path.join(paths['IMAGE_PATH_VIDEO'], '1_out.mp4')

print(VIDEO_FILE_IN)
print(VIDEO_FILE_OUT)

In [ ]:
cap = cv2.VideoCapture(VIDEO_FILE_IN)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(VIDEO_FILE_OUT, cv2.VideoWriter_fourcc('M','J','P','G'), 60, (width, height))

while True:
    ret, frame = cap.read()
    
    if ret == True:
        image_np = np.array(frame)
    
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
    
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.75,
                agnostic_mode=False)

        out.write(image_np_with_detections)
    
        cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()
        

# 12. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

# 13. Conversion to TFJS

In [ ]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

In [ ]:
# Test Code: https://github.com/nicknochnack/RealTimeSignLanguageDetectionwithTFJS

# 14. Conversion to TFLite

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

In [ ]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [ ]:
print(command)

numpy was 1.18.5 now 1.19.2
h5py was 2.10.0 now 3.1.0
gast was 0.3.3 now 0.4.0

In [ ]:
!{command}

# 13. Zip and Export Models 

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}